# Sport Articles - Semantic Search

## Objectives:
1. Convert raw text into dense vector representations
2. Build and query a vector index (FAISS)
3. Evaluate semantic similarity between queries and documents

## Pipeline:
```
Text → Sliding Window Chunking → Encoder → Vectors → FAISS Index → Query → Ranked Results
```

## Key Features:
- **Sliding Window Chunking**: Split articles into overlapping chunks (500 chars, 100 overlap)
- **Multilingual E5**: Dense encoder supporting Indonesian language
- **FAISS IndexFlatIP**: Exact cosine similarity search

## Step 1: Load and Prepare Data

Load the cleaned articles from `detik_sport_articles_cleaned.json`

In [1]:
import json
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# Load cleaned articles
DATA_PATH = "scraping_result/detik_sport_articles_cleaned.json"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    articles = json.load(f)

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(articles)
print(f"Loaded {len(df)} articles")
print(f"\nColumns: {df.columns.tolist()}")

# Validate required columns
required_columns = ['title', 'content', 'date', 'url']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"⚠️ Missing columns: {missing_columns}")
    # Create placeholder columns if missing
    for col in missing_columns:
        if col == 'title':
            df['title'] = df['content'].str[:50] + "..."
        elif col == 'date':
            df['date'] = "Unknown"
        elif col == 'url':
            df['url'] = ""
else:
    print("✅ All required columns present")

# Remove rows with empty or very short content
initial_count = len(df)
df = df[df['content'].str.len() > 50].reset_index(drop=True)
print(f"Removed {initial_count - len(df)} articles with very short content")
print(f"Final dataset: {len(df)} articles")

print(f"\nSample article:")
print(f"Title: {df.iloc[0]['title']}")
print(f"Content preview: {df.iloc[0]['content'][:300]}...")

Loaded 397 articles

Columns: ['url', 'title', 'date', 'author', 'content']
✅ All required columns present
Removed 1 articles with very short content
Final dataset: 396 articles

Sample article:
Title: Deco: Rashford Menderita di MU
Content preview: Jakarta - Performa Marcus Rashford membaik di Barcelona . Direktur Barcelona Deco mengungkap penyebab Rashford kesulitan di Manchester United . Rashford dipinjamkan MU ke Barcelona pada musim panas lalu. Penyerang Inggris itu dilepas usai tidak masuk ke rencana Ruben Amorim. Ini jadi kali kedua seca...


## Step 1b: Sliding Window Chunking

Split long articles into overlapping chunks to:
- **Preserve detailed context** within each chunk
- **Maintain continuity** via overlap between consecutive chunks
- **Improve retrieval precision** for specific facts

**Parameters:**
- `chunk_size`: Maximum characters per chunk (default: 500)
- `overlap`: Characters shared between consecutive chunks (default: 100)

```
Article: [===========================================]
Chunk 1: [========]
Chunk 2:      [========]  ← overlaps with chunk 1
Chunk 3:           [========]  ← overlaps with chunk 2
```

In [2]:
def sliding_window_chunking(
    text: str,
    chunk_size: int = 500,
    overlap: int = 100
) -> list[str]:
    """
    Split text into overlapping chunks using sliding window.

    Args:
        text: Input text to chunk
        chunk_size: Maximum characters per chunk
        overlap: Characters shared between consecutive chunks

    Returns:
        List of text chunks
    """
    if len(text) <= chunk_size:
        return [text]

    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]

        # Try to break at sentence boundary (. ! ?)
        if end < len(text):
            # Look for last sentence boundary in the chunk
            last_period = max(
                chunk.rfind('. '),
                chunk.rfind('! '),
                chunk.rfind('? ')
            )
            if last_period > chunk_size // 2:  # Only if it's past halfway
                chunk = chunk[:last_period + 1]
                end = start + last_period + 1

        chunks.append(chunk.strip())

        # Move window forward (chunk_size - overlap)
        start = end - overlap

        # Avoid tiny final chunks
        if len(text) - start < overlap:
            break

    return chunks


def chunk_articles(
    df: pd.DataFrame,
    chunk_size: int = 500,
    overlap: int = 100
) -> pd.DataFrame:
    """
    Apply sliding window chunking to all articles.

    Returns DataFrame with columns:
    - article_id: Original article index
    - chunk_id: Chunk index within article
    - title: Article title (same for all chunks)
    - content: Chunk content
    - date, url: Original metadata
    """
    chunks_data = []

    for idx, row in df.iterrows():
        full_text = row['content']
        chunks = sliding_window_chunking(full_text, chunk_size, overlap)

        for chunk_idx, chunk_text in enumerate(chunks):
            chunks_data.append({
                'article_id': idx,
                'chunk_id': chunk_idx,
                'title': row['title'],
                'content': chunk_text,
                'date': row['date'],
                'url': row['url'],
                'total_chunks': len(chunks)
            })

    return pd.DataFrame(chunks_data)


# Apply chunking with parameters
CHUNK_SIZE = 500  # characters per chunk
OVERLAP = 100     # overlap between chunks

print(f"📐 Chunking Parameters:")
print(f"   - Chunk size: {CHUNK_SIZE} chars")
print(f"   - Overlap: {OVERLAP} chars")
print(f"   - Effective step: {CHUNK_SIZE - OVERLAP} chars")
print()

# Create chunked dataset
df_chunks = chunk_articles(df, chunk_size=CHUNK_SIZE, overlap=OVERLAP)

print(f"📊 Chunking Results:")
print(f"   - Original articles: {len(df)}")
print(f"   - Total chunks: {len(df_chunks)}")
print(f"   - Average chunks per article: {len(df_chunks) / len(df):.1f}")
print()

# Show chunk distribution
chunk_counts = df_chunks.groupby('article_id')['chunk_id'].count()
print(f"📈 Chunk Distribution:")
print(f"   - Min chunks: {chunk_counts.min()}")
print(f"   - Max chunks: {chunk_counts.max()}")
print(f"   - Articles with 1 chunk: {(chunk_counts == 1).sum()}")
print(f"   - Articles with 2+ chunks: {(chunk_counts >= 2).sum()}")

📐 Chunking Parameters:
   - Chunk size: 500 chars
   - Overlap: 100 chars
   - Effective step: 400 chars

📊 Chunking Results:
   - Original articles: 396
   - Total chunks: 2325
   - Average chunks per article: 5.9

📈 Chunk Distribution:
   - Min chunks: 2
   - Max chunks: 22
   - Articles with 1 chunk: 0
   - Articles with 2+ chunks: 396


In [3]:
# Visualize chunking on a sample article
sample_idx = df[df['content'].str.len() > 1000].index[0]  # Pick a long article
sample_article = df.iloc[sample_idx]

print(f"📰 Sample Article: {sample_article['title']}")
print(f"   Total length: {len(sample_article['content'])} chars")
print("=" * 80)

sample_chunks = df_chunks[df_chunks['article_id'] == sample_idx]
print(f"\n🔪 Split into {len(sample_chunks)} chunks:\n")

for _, chunk_row in sample_chunks.iterrows():
    chunk_num = chunk_row['chunk_id'] + 1
    content = chunk_row['content']
    print(f"━━━ Chunk {chunk_num}/{chunk_row['total_chunks']} ({len(content)} chars) ━━━")
    print(content[:200] + "..." if len(content) > 200 else content)
    print()

📰 Sample Article: Deco: Rashford Menderita di MU
   Total length: 1250 chars

🔪 Split into 4 chunks:

━━━ Chunk 1/4 (447 chars) ━━━
Jakarta - Performa Marcus Rashford membaik di Barcelona . Direktur Barcelona Deco mengungkap penyebab Rashford kesulitan di Manchester United . Rashford dipinjamkan MU ke Barcelona pada musim panas la...

━━━ Chunk 2/4 (453 chars) ━━━
umnya, pesepakbola berusia 28 tahun itu dipinjamkna ke Aston Villa pada paruh kedua musim 2024/2025. Bersama Barcelona, Rashford perlahan menemukan kembali performanya. Sejauh ini, ia sudah mencetak e...

━━━ Chunk 3/4 (383 chars) ━━━
ani. "Rashford bahagia bersama kami. Marcus adalah pemain luar biasa," ujar Deco kepada Times Sport. "Dia juga sedikit menderita dengan perubahan generasi di United. Itu tidak mudah. Ketika Anda adala...

━━━ Chunk 4/4 (266 chars) ━━━
lima tahun terakhir, mereka kesulitan membangun kembali tim, agar lebih kuat lagi. Dia ada di sana. Jadi tidak mudah bagi seorang pemain yang dapat banyak tuntutan

## Step 2: Dense Encoding with Multilingual E5

Using `intfloat/multilingual-e5-base` because:
- ✅ Supports 100+ languages including **Indonesian**
- ✅ State-of-the-art performance for multilingual semantic search
- ✅ Good balance between accuracy and speed

**Note:** E5 models require a prefix for the input:
- For documents/passages: `"passage: "` + text
- For queries: `"query: "` + text

In [4]:
# Disable tqdm widget to avoid ipywidget rendering issues in VS Code
os.environ["TQDM_DISABLE"] = "0"  # Keep tqdm but use text mode

from sentence_transformers import SentenceTransformer
import logging

# Suppress unnecessary warnings
logging.getLogger("sentence_transformers").setLevel(logging.WARNING)

# Load the multilingual E5 model
MODEL_NAME = "intfloat/multilingual-e5-base"
print(f"Loading model: {MODEL_NAME}")

model = SentenceTransformer(MODEL_NAME, device="cpu")  # Explicitly set device
print(f"Model loaded successfully!")
print(f"Embedding dimension: {model.get_sentence_embedding_dimension()}")

Loading model: intfloat/multilingual-e5-base
Model loaded successfully!
Embedding dimension: 768
Model loaded successfully!
Embedding dimension: 768


In [6]:
# Prepare CHUNKS for encoding (not full documents)
# E5 models require "passage: " prefix for documents

def prepare_chunks_for_encoding(df_chunks: pd.DataFrame) -> list[str]:
    """
    Prepare chunks for E5 encoding.
    Combines title and chunk content with the required prefix.
    """
    documents = []
    for _, row in df_chunks.iterrows():
        # Combine title and chunk content for richer representation
        text = f"{row['title']}. {row['content']}"
        # Add E5 passage prefix
        documents.append(f"passage: {text}")
    return documents

# Prepare all chunks
documents = prepare_chunks_for_encoding(df_chunks)
print(f"Prepared {len(documents)} chunks for encoding")
print(f"(from {len(df)} original articles)")

Prepared 2325 chunks for encoding
(from 396 original articles)


In [7]:
# Encode all documents into dense vectors
print("Encoding documents... (this may take a few minutes)")

# Encode with text-based progress bar (not widget)
embeddings = model.encode(
    documents,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # Normalize for cosine similarity
)

print(f"\nEncoding complete!")
print(f"Embeddings shape: {embeddings.shape}")
print(f"Each document is represented by a {embeddings.shape[1]}-dimensional vector")

Encoding documents... (this may take a few minutes)


Batches:   0%|          | 0/73 [00:00<?, ?it/s]


Encoding complete!
Embeddings shape: (2325, 768)
Each document is represented by a 768-dimensional vector


In [8]:
# Save embeddings and chunk metadata for later use
EMBEDDINGS_PATH = "embedding/article_embeddings.npy"
CHUNKS_PATH = "embedding/chunks_metadata.json"

# Save embeddings
np.save(EMBEDDINGS_PATH, embeddings)
print(f"✅ Embeddings saved to {EMBEDDINGS_PATH}")
print(f"   File size: {embeddings.nbytes / 1024 / 1024:.2f} MB")

# Save chunk metadata (needed for retrieval)
chunks_metadata = df_chunks.to_dict(orient='records')
with open(CHUNKS_PATH, 'w', encoding='utf-8') as f:
    json.dump(chunks_metadata, f, ensure_ascii=False, indent=2)
print(f"✅ Chunk metadata saved to {CHUNKS_PATH}")
print(f"   Total chunks: {len(chunks_metadata)}")

✅ Embeddings saved to embedding/article_embeddings.npy
   File size: 6.81 MB
✅ Chunk metadata saved to embedding/chunks_metadata.json
   Total chunks: 2325


## Step 3: Building FAISS Vector Index

FAISS (Facebook AI Similarity Search) is a library for efficient similarity search and clustering of dense vectors.

In [9]:
import faiss

# Get embedding dimension
dimension = embeddings.shape[1]

# Create FAISS index
# Using IndexFlatIP for Inner Product (cosine similarity with normalized vectors)
index = faiss.IndexFlatIP(dimension)

# Add embeddings to the index
index.add(embeddings.astype('float32'))

print(f"FAISS index created!")
print(f"Index type: Flat Inner Product (exact search)")
print(f"Dimension: {dimension}")
print(f"Total vectors indexed: {index.ntotal}")

# Save FAISS index for later use
INDEX_PATH = "faiss/faiss_index.bin"
faiss.write_index(index, INDEX_PATH)
print(f"\n💾 FAISS index saved to {INDEX_PATH}")
print(f"File size: {os.path.getsize(INDEX_PATH) / 1024 / 1024:.2f} MB")

FAISS index created!
Index type: Flat Inner Product (exact search)
Dimension: 768
Total vectors indexed: 2325

💾 FAISS index saved to faiss/faiss_index.bin
File size: 6.81 MB


### Optional: Load Existing Embeddings and Index

Run this cell instead of Step 2 & 3 if you already have saved embeddings and index.

In [10]:
# Skip encoding if embeddings and index already exist
# Run this cell INSTEAD of Step 1b, 2 & 3 encoding cells if you want to reload existing data

import faiss
from sentence_transformers import SentenceTransformer

EMBEDDINGS_PATH = "embedding/article_embeddings.npy"
CHUNKS_PATH = "embedding/chunks_metadata.json"
INDEX_PATH = "faiss/faiss_index.bin"
MODEL_NAME = "intfloat/multilingual-e5-base"

if os.path.exists(EMBEDDINGS_PATH) and os.path.exists(INDEX_PATH) and os.path.exists(CHUNKS_PATH):
    print("📂 Found existing embeddings, chunks, and index. Loading...")

    # Load embeddings
    embeddings = np.load(EMBEDDINGS_PATH)

    # Load FAISS index
    index = faiss.read_index(INDEX_PATH)

    # Load chunk metadata
    with open(CHUNKS_PATH, 'r', encoding='utf-8') as f:
        chunks_metadata = json.load(f)
    df_chunks = pd.DataFrame(chunks_metadata)

    # Load model for query encoding
    print(f"Loading model: {MODEL_NAME}")
    model = SentenceTransformer(MODEL_NAME)

    print(f"✅ Loaded embeddings: {embeddings.shape}")
    print(f"✅ Loaded FAISS index: {index.ntotal} vectors")
    print(f"✅ Loaded chunk metadata: {len(df_chunks)} chunks")
    print(f"✅ Model loaded for query encoding")
else:
    missing = []
    if not os.path.exists(EMBEDDINGS_PATH): missing.append("embeddings")
    if not os.path.exists(INDEX_PATH): missing.append("FAISS index")
    if not os.path.exists(CHUNKS_PATH): missing.append("chunk metadata")
    print(f"❌ Missing: {', '.join(missing)}. Please run Step 1b, 2 & 3 cells to encode documents.")

📂 Found existing embeddings, chunks, and index. Loading...
Loading model: intfloat/multilingual-e5-base
✅ Loaded embeddings: (2325, 768)
✅ Loaded FAISS index: 2325 vectors
✅ Loaded chunk metadata: 2325 chunks
✅ Model loaded for query encoding
✅ Loaded embeddings: (2325, 768)
✅ Loaded FAISS index: 2325 vectors
✅ Loaded chunk metadata: 2325 chunks
✅ Model loaded for query encoding


## Step 4: Query and Retrieval

Create a search function that:
1. Encodes the query using the same model
2. Uses FAISS to find top-k similar documents
3. Returns titles and similarity scores

In [11]:
def search(query: str, top_k: int = 5, deduplicate: bool = True) -> pd.DataFrame:
    """
    Search for semantically similar chunks.

    Args:
        query: Search query in Indonesian or English
        top_k: Number of results to return
        deduplicate: If True, return only best chunk per article

    Returns:
        DataFrame with search results
    """
    # E5 requires "query: " prefix for queries
    query_with_prefix = f"query: {query}"

    # Encode the query
    query_embedding = model.encode(
        [query_with_prefix],
        normalize_embeddings=True,
        convert_to_numpy=True
    )

    # Search in FAISS index (get more results if deduplicating)
    search_k = top_k * 3 if deduplicate else top_k
    scores, indices = index.search(query_embedding.astype('float32'), search_k)

    # Build results from chunks
    results = []
    seen_articles = set()

    for score, idx in zip(scores[0], indices[0]):
        chunk = df_chunks.iloc[idx]
        article_id = chunk['article_id']

        # Skip if we already have a chunk from this article (deduplication)
        if deduplicate and article_id in seen_articles:
            continue
        seen_articles.add(article_id)

        results.append({
            'rank': len(results) + 1,
            'score': float(score),
            'title': chunk['title'],
            'date': chunk['date'],
            'chunk_id': f"{article_id}_{chunk['chunk_id']}",
            'chunk_num': f"{chunk['chunk_id']+1}/{chunk['total_chunks']}",
            'content_preview': chunk['content'][:250] + "..." if len(chunk['content']) > 250 else chunk['content'],
            'full_content': chunk['content'],
            'url': chunk['url'],
            'article_id': article_id
        })

        if len(results) >= top_k:
            break

    return pd.DataFrame(results)


def display_results(query: str, top_k: int = 5, deduplicate: bool = True):
    """Pretty print search results"""
    print(f"🔍 Query: \"{query}\"")
    print("=" * 80)

    results = search(query, top_k, deduplicate)

    for _, row in results.iterrows():
        print(f"\n#{row['rank']} | Score: {row['score']:.4f} | Chunk: {row['chunk_num']}")
        print(f"📰 {row['title']}")
        print(f"📅 {row['date']}")
        print(f"📝 {row['content_preview']}")
        print("-" * 80)

    return results

## Step 5: Qualitative Evaluation

Test the semantic search with various queries in Indonesian to see if the results are semantically related.

In [12]:
# Test Query 1: Tentang pemain sepakbola tertentu
results1 = display_results("Ronaldo Piala Dunia 2026", top_k=5)

🔍 Query: "Ronaldo Piala Dunia 2026"

#1 | Score: 0.8770 | Chunk: 1/3
📰 Ronaldo Lolos Sanksi, Bisa Main di Fase Grup Piala Dunia 2026
📅 Rabu, 26 Nov 2025 07:00 WIB
📝 Zurich - Timnas Portugal bisa bernapas lega. Sebab Cristiano Ronaldo lolos dari sanksi kartu merah dan bisa main di Piala Dunia 2026 . Sebelumnya, Ronaldo dikartumerah saat Portugal tumbang 0-2 di kandang Republik Irlandia, karena menyikut bek lawan ...
--------------------------------------------------------------------------------

#2 | Score: 0.8729 | Chunk: 2/6
📰 Netizen 'Ngamuk' Usai FIFA Ringankan Hukuman Ronaldo di Piala Dunia 2026
📅 Rabu, 26 Nov 2025 10:08 WIB
📝 obaan. Dari tiga laga, satu pertandingan sudah dilewatkan Ronaldo ketika Portugal menggilas Armenia. Dua laga berikutnya, yang harusnya dimainkan Portugal di Piala Dunia 2026, CR7 tetap bisa tampil. Sanksi dua laga berikutnya ditangguhkan FIFA untuk ...
--------------------------------------------------------------------------------

#3 | Score: 0.8615 | Chu

In [13]:
# Test Query 2: Tentang liga tertentu
results2 = display_results("Klasemen Liga Inggris terbaru", top_k=5)

🔍 Query: "Klasemen Liga Inggris terbaru"

#1 | Score: 0.8444 | Chunk: 3/5
📰 Jadwal Liga Inggris Tengah Pekan Ini: Man City Main Nanti Malam
📅 Selasa, 02 Des 2025 09:40 WIB
📝 ujian dari Sunderland. The Reds akan menjamu Granit Xhaka dkk. di Anfield pada Kamis dini hari WIB. Liverpool baru saja memutus tren negatifnya dengan kemenangan 2-0 di markas West Ham United akhir pekan lalu. Menilik klasemen Liga Inggris, Liverpool...
--------------------------------------------------------------------------------

#2 | Score: 0.8398 | Chunk: 1/4
📰 Pemain dengan Assist Terbanyak di Liga Inggris
📅 Selasa, 02 Des 2025 14:40 WIB
📝 Daftar Isi Daftar Pemain dengan Assist Terbanyak di Liga Inggris Jakarta - Liga Inggris, atau yang biasa dikenal sebagai Premier League, merupakan liga sepakbola yang amat populer di dunia. Terdapat 51 klub yang sudah berkompetisi di liga ini sejak d...
--------------------------------------------------------------------------------

#3 | Score: 0.8390 | Chunk: 4/5
📰 Klase

In [14]:
# Test Query 3: Tentang Timnas Indonesia
results3 = display_results("Pelatih baru Timnas Indonesia", top_k=5)

🔍 Query: "Pelatih baru Timnas Indonesia"

#1 | Score: 0.8488 | Chunk: 1/2
📰 Gabung Navbahor, Kapadze Dipastikan Tidak Latih Timnas Indonesia
📅 Selasa, 02 Des 2025 00:00 WIB
📝 Jakarta - Timur Kapadze dipastikan tidak akan menjadi pelatih Timnas Indonesia . Dia sudah menerima pinangan klub Navbahor Namangan. "Navbahor hari ini secara resmi memperkenalkan Timur Kapadze sebagai pelatih kepala baru. Kedua pihak telah menandata...
--------------------------------------------------------------------------------

#2 | Score: 0.8412 | Chunk: 7/7
📰 SEA Games 2025: Demi Quattrick Emas, Timnas Voli RI ke China
📅 Minggu, 16 Nov 2025 12:50 WIB
📝 fal Mauluddani Kristoforus Sina Jasen Natanael Kilanta Alfin Daniel Pratama Prasojo Fahreza Rakha Abhinaya Ofisial: Jeff Jiang , Erwin Rusni, Novie Efendi Lestari Mulyono, Setiawan Adi Praja Kusuma Nur Amanu, Aditya Pahlevi Manajer: Adang Ginanjar se...
--------------------------------------------------------------------------------

#3 | Score: 0.8325 | Chu

In [15]:
# Test Query 4: Tentang transfer pemain
results4 = display_results("Barcelona beli pemain baru", top_k=5)

🔍 Query: "Barcelona beli pemain baru"

#1 | Score: 0.8231 | Chunk: 1/3
📰 Hari Sempurna buat Barcelona: Main Lagi di Camp Nou, Menang, Clean Sheet
📅 Minggu, 23 Nov 2025 11:00 WIB
📝 Jakarta - Barcelona menandai comeback-nya ke Camp Nou dengan kemenangan telak atas Athletic Bilbao . Segalanya terasa sempurna untuk Blaugrana. Barcelona menjamu Bilbao di Camp Nou dalam lanjutan LaLiga , Sabtu malam WIB. Ini jadi pertandingan pertam...
--------------------------------------------------------------------------------

#2 | Score: 0.8210 | Chunk: 1/8
📰 Barcelona Vs Bilbao: Blaugrana Menang 4-0, Naik ke Puncak Klasemen
📅 Minggu, 23 Nov 2025 00:33 WIB
📝 Jakarta - Barcelona menang 4-0 atas Athletic Bilbao dalam pertandingan di Camp Nou. Hasil ini membawa Barcelona menggeser Real Madrid dari puncak klasemen. Laga Barcelona vs Bilbao di pekan ke-13 LaLiga digelardi Camp Nou, Sabtu malam WIB. Blaugrana ...
--------------------------------------------------------------------------------

#3 | Score: 0

In [17]:
# Test Query 5: Query dalam bahasa Inggris (test multilingual capability)
results5 = display_results("juara dunia badminton", top_k=5)

🔍 Query: "juara dunia badminton"

#1 | Score: 0.8295 | Chunk: 6/8
📰 Atlet Muda RI Sukses Kuasai Podium Wondr by BNI International Challenge
📅 Selasa, 25 Nov 2025 12:16 WIB
📝 i konkret bagi lahirnya bintang-bintang baru yang siap menjaga dominasi Indonesia di panggung dunia. Panen gelar di Yogyakarta menjadi bukti bahwa masa depan bulutangkis Indonesia berada di tangan generasi muda yang telah terbina dengan baik dan siap...
--------------------------------------------------------------------------------

#2 | Score: 0.8231 | Chunk: 2/3
📰 Perbati Jadi Anggota World Boxing
📅 Senin, 24 Nov 2025 02:30 WIB
📝 Kongres World Boxing," kata Ketua Umum Perbati, Ray Zulham Farras Nugraha, dalam keterangan persnya. Ketua Ketua Umum Komite Olimpiade Indonesia , Raja Sapta Oktohari, sempat hadir dalam kongres tersebut. Bersama Okto, Ray bertemu Gennady Golovkin se...
--------------------------------------------------------------------------------

#3 | Score: 0.8196 | Chunk: 1/6
📰 Kisah Ralyya, Pebul

## Step 5b: Quantitative Evaluation

Evaluate search quality with predefined test cases using **Precision@K** metric.

In [18]:
def evaluate_search_quality(test_queries: list[dict], top_k: int = 5) -> pd.DataFrame:
    """
    Evaluate search quality with predefined test cases.

    Args:
        test_queries: List of dicts with 'query' and 'expected_keywords'
        top_k: Number of results to evaluate

    Returns:
        DataFrame with evaluation metrics
    """
    results = []

    for test in test_queries:
        query = test['query']
        expected_keywords = test['expected_keywords']

        search_results = search(query, top_k=top_k, deduplicate=True)

        # Calculate hits: how many results contain at least one expected keyword
        hits = 0
        for _, row in search_results.iterrows():
            title_content = (row['title'] + " " + row['full_content']).lower()
            if any(kw.lower() in title_content for kw in expected_keywords):
                hits += 1

        precision_at_k = hits / top_k

        results.append({
            'query': query,
            'expected_keywords': ', '.join(expected_keywords[:3]) + '...',
            f'hits@{top_k}': hits,
            f'precision@{top_k}': precision_at_k,
            'top_score': search_results.iloc[0]['score'] if len(search_results) > 0 else 0,
            'avg_score': search_results['score'].mean() if len(search_results) > 0 else 0
        })

    return pd.DataFrame(results)

In [19]:
# Define test cases with expected keywords
test_queries = [
    {
        'query': 'Ronaldo gol Piala Dunia',
        'expected_keywords': ['ronaldo', 'cristiano', 'cr7', 'gol', 'piala dunia']
    },
    {
        'query': 'Liga Inggris klasemen',
        'expected_keywords': ['premier league', 'liga inggris', 'klasemen', 'epl', 'inggris']
    },
    {
        'query': 'Timnas Indonesia pelatih',
        'expected_keywords': ['timnas', 'indonesia', 'pelatih', 'garuda', 'pssi']
    },
    {
        'query': 'Barcelona transfer pemain',
        'expected_keywords': ['barcelona', 'barca', 'transfer', 'beli', 'blaugrana']
    },
    {
        'query': 'MotoGP balapan juara',
        'expected_keywords': ['motogp', 'motor', 'juara', 'race', 'gp', 'balapan']
    },
    {
        'query': 'Liga Italia Serie A',
        'expected_keywords': ['serie a', 'liga italia', 'italia', 'inter', 'milan', 'juventus']
    },
    {
        'query': 'Manchester United masalah',
        'expected_keywords': ['manchester', 'united', 'mu', 'man utd', 'old trafford']
    }
]

# Run evaluation
eval_results = evaluate_search_quality(test_queries, top_k=5)

# Display results
print("📊 Quantitative Evaluation Results")
print("=" * 90)
print(eval_results.to_string(index=False))
print("\n" + "=" * 90)
print(f"📈 Average Precision@5: {eval_results['precision@5'].mean():.2%}")
print(f"📈 Average Top Score: {eval_results['top_score'].mean():.4f}")
print(f"📈 Average Score: {eval_results['avg_score'].mean():.4f}")

📊 Quantitative Evaluation Results
                    query                         expected_keywords  hits@5  precision@5  top_score  avg_score
  Ronaldo gol Piala Dunia                ronaldo, cristiano, cr7...       5          1.0   0.856364   0.845966
    Liga Inggris klasemen premier league, liga inggris, klasemen...       5          1.0   0.843397   0.830476
 Timnas Indonesia pelatih             timnas, indonesia, pelatih...       5          1.0   0.842489   0.833422
Barcelona transfer pemain             barcelona, barca, transfer...       5          1.0   0.828815   0.820697
     MotoGP balapan juara                   motogp, motor, juara...       5          1.0   0.841939   0.838929
      Liga Italia Serie A           serie a, liga italia, italia...       4          0.8   0.824424   0.807094
Manchester United masalah                 manchester, united, mu...       5          1.0   0.841527   0.835348

📈 Average Precision@5: 97.14%
📈 Average Top Score: 0.8399
📈 Average Score: 0.

## Summary Statistics

Overview of the semantic search system.

In [20]:
INDEX_PATH = "faiss/faiss_index.bin"
EMBEDDINGS_PATH = "embedding/article_embeddings.npy"
CHUNKS_PATH = "embedding/chunks_metadata.json"

print("📈 Semantic Search System Statistics")
print("=" * 50)
print(f"📚 Original articles: {len(df)}")
print(f"🔪 Total chunks indexed: {len(df_chunks)}")
print(f"📊 Avg chunks per article: {len(df_chunks) / len(df):.1f}")
print(f"📐 Embedding dimension: {embeddings.shape[1]}")
print(f"🤖 Model: intfloat/multilingual-e5-base")
print(f"🔍 Index type: FAISS Flat Inner Product")
print(f"\n⚙️ Chunking Parameters:")
print(f"   - Chunk size: {CHUNK_SIZE} chars")
print(f"   - Overlap: {OVERLAP} chars")
print(f"\n💾 Storage:")
print(f"   - Embeddings: {embeddings.nbytes / 1024 / 1024:.2f} MB")
if os.path.exists(INDEX_PATH):
    print(f"   - FAISS index: {os.path.getsize(INDEX_PATH) / 1024 / 1024:.2f} MB")
if os.path.exists(CHUNKS_PATH):
    print(f"   - Chunk metadata: {os.path.getsize(CHUNKS_PATH) / 1024:.1f} KB")
print(f"\n📊 Evaluation:")
print(f"   - Average Precision@5: {eval_results['precision@5'].mean():.2%}")
print(f"   - Best performing query: {eval_results.loc[eval_results['precision@5'].idxmax(), 'query']}")
print(f"   - Worst performing query: {eval_results.loc[eval_results['precision@5'].idxmin(), 'query']}")

📈 Semantic Search System Statistics
📚 Original articles: 396
🔪 Total chunks indexed: 2325
📊 Avg chunks per article: 5.9
📐 Embedding dimension: 768
🤖 Model: intfloat/multilingual-e5-base
🔍 Index type: FAISS Flat Inner Product

⚙️ Chunking Parameters:
   - Chunk size: 500 chars
   - Overlap: 100 chars

💾 Storage:
   - Embeddings: 6.81 MB
   - FAISS index: 6.81 MB
   - Chunk metadata: 1703.7 KB

📊 Evaluation:
   - Average Precision@5: 97.14%
   - Best performing query: Ronaldo gol Piala Dunia
   - Worst performing query: Liga Italia Serie A
